In [1]:
# Configuración
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [3]:
#leer archivos
df = pd.read_excel('/Users/agustin/Documents/Agustin_2025/Juan Colon/analisis.ipynb/ACTUALES LESIONES PLANTEL 2026.xlsx')
df.head()

,JUGADOR,FECHA,FECHA DE ALTA,TIPO DE LESION,REGION,DIAGNOSTICO,LATERALIDAD,DIAS FUERA
0,CASTET FACUNDO,2024-01-18,2024-01-20,MUSCULOTENDINOSA,ISQUIOTIBIALES,EDEMA INTERFIBRILAR BICEPS FEMORAL DERECHO,DERECHO,2
1,GIMENEZ TOMAS,2024-01-11,2024-02-06,ARTICULAR,RODILLA,RUPTURA EN ASA DE BALDE DEL MENISCO INTERNO DE...,IZQUIERDO,26
2,CASTET FACUNDO,2024-03-06,2024-03-07,ARTICULAR,RODILLA,MINIMO EDEMA INTERFIBRILAR Y PERI FASCIAL A NI...,IZQUIERDO,1
3,CASTET FACUNDO,2024-10-07,2024-11-08,ARTICULAR,TOBILLO,"ESGUINCE SEVERO PAA, ESGUINCE MODERADO CON RUP...",DERECHO,32
4,GIMENEZ AGUSTIN,2024-11-12,2024-11-19,OTRA,PIE,"EDEMA EN DIAFISIS DEL 2DO, 3ER Y 4TO MTT",DERECHO,7


In [18]:
# Control rápido de estructura esperada
expected_cols = {'jugador', 'fecha', 'fecha_de_alta', 'tipo_de_lesion',
                 'region', 'diagnostico', 'lateralidad', 'dias_fuera'}

current_cols = set(df.columns)
if not expected_cols.issubset(current_cols):
    raise ValueError(f"⚠️ Columnas inesperadas. El archivo tiene: {list(df.columns)}")
else:
    print("✅ Estructura del archivo correcta, podés continuar.")


✅ Estructura del archivo correcta, podés continuar.


In [4]:
# Ver filas y columnas
df.shape

(58, 8)

In [5]:
# Limpiar nombres de columnas
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")     
    .str.replace("á", "a")
    .str.replace("é", "e")
    .str.replace("í", "i")
    .str.replace("ó", "o")
    .str.replace("ú", "u")
)
df.columns

Index(['jugador', 'fecha', 'fecha_de_alta', 'tipo_de_lesion', 'region',
       'diagnostico', 'lateralidad', 'dias_fuera'],
      dtype='object')

In [6]:
# Tipo de datos y valores faltantes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58 entries, 0 to 57
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   jugador         42 non-null     object        
 1   fecha           42 non-null     datetime64[ns]
 2   fecha_de_alta   36 non-null     datetime64[ns]
 3   tipo_de_lesion  42 non-null     object        
 4   region          42 non-null     object        
 5   diagnostico     42 non-null     object        
 6   lateralidad     42 non-null     object        
 7   dias_fuera      58 non-null     int64         
dtypes: datetime64[ns](2), int64(1), object(5)
memory usage: 3.8+ KB


In [7]:
# Valores faltantes por columnas
df.isnull().sum()

jugador           16
fecha             16
fecha_de_alta     22
tipo_de_lesion    16
region            16
diagnostico       16
lateralidad       16
dias_fuera         0
dtype: int64

In [8]:
# Ver filas con valores faltantes
df[df.isnull().any(axis=1)]

,jugador,fecha,fecha_de_alta,tipo_de_lesion,region,diagnostico,lateralidad,dias_fuera
24,BETTINI GONZALO,2025-09-22,NaT,MUSCULOTENDINOSA,CUADRICEPS,3A RAC DER,DERECHO,-45922
25,CASTRO FACUNDO,2025-09-22,NaT,OTRA,PIE,ESPOLON CALCANEO + FASCITIS PLANTAR (IZQ),IZQUIERDO,-45922
26,GALLAY TOMAS,2025-09-05,NaT,MUSCULOTENDINOSA,ISQUIOTIBIALES,2B SEMIMEMBRANOSO PROXIMAL IZQ,IZQUIERDO,-45905
27,GIMENEZ AGUSTIN,2025-07-25,NaT,MUSCULOTENDINOSA,CUADRICEPS,2C RAC PROXIMAL DER,DERECHO,-45863
28,IBARRA CONRADO,2025-10-15,NaT,MUSCULOTENDINOSA,PIE,EDEMA DE PARTES BLANDAS DORSALES DEL MEDIOPIE,DERECHO,-45945
29,SOTO GONZALO,2025-09-27,NaT,MUSCULOTENDINOSA,ADUCTORES,2B ADUCTOR LARGO IZQ,IZQUIERDO,-45927
42,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0
43,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0
44,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0
45,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0


In [9]:
# Cuantos jugadores distintos tengo
df['jugador'].nunique()

20

In [10]:
# Nombre de esos jugadores
df['jugador'].unique()

array(['CASTET FACUNDO', 'GIMENEZ TOMAS', 'GIMENEZ AGUSTIN',
       'THALLER NICOLAS', 'LAGO IGNACIO', 'TALPONE NICOLAS',
       'GARCIA CRISTIAN', 'CASTRO FACUNDO', 'RODRIGUEZ LUIS MIGUEL',
       'SOTO GONZALO', 'GIGLIOTTI EMMANUEL', 'BETTINI GONZALO',
       'GALLAY TOMAS', 'IBARRA CONRADO', 'YUNIS ZAHIR',
       'BERNARDI CRISTIAN', 'JOURDAN FEDERICO', 'SANCHEZ FACUNDO',
       'gaitan lautaro', 'GAITAN LAUTARO', nan], dtype=object)

In [14]:
# Copia de seguridad
df['jugador_original'] = df['jugador']

In [15]:
# Normaliar formato
df['jugador'] = df['jugador'].str.title().str.strip()

In [16]:
# Como quedo el df
df['jugador'].unique()

array(['Castet Facundo', 'Gimenez Tomas', 'Gimenez Agustin',
       'Thaller Nicolas', 'Lago Ignacio', 'Talpone Nicolas',
       'Garcia Cristian', 'Castro Facundo', 'Rodriguez Luis Miguel',
       'Soto Gonzalo', 'Gigliotti Emmanuel', 'Bettini Gonzalo',
       'Gallay Tomas', 'Ibarra Conrado', 'Yunis Zahir',
       'Bernardi Cristian', 'Jourdan Federico', 'Sanchez Facundo',
       'Gaitan Lautaro', nan], dtype=object)

In [17]:
# Jugadores con mas de una lesion
conteo = df['jugador'].value_counts()
conteo[conteo > 1]

jugador
Castet Facundo           5
Lago Ignacio             4
Ibarra Conrado           4
Rodriguez Luis Miguel    3
Gimenez Agustin          3
Thaller Nicolas          3
Bernardi Cristian        2
Gimenez Tomas            2
Soto Gonzalo             2
Castro Facundo           2
Garcia Cristian          2
Talpone Nicolas          2
Gaitan Lautaro           2
Name: count, dtype: int64

In [19]:
# Revisar valores de dias de baja
today = pd.Timestamp(datetime.today().date())

# Calcular días de baja correctamente
df["dias_fuera"] = (
    df["fecha_de_alta"].fillna(today) - df["fecha"]
).dt.days

# Si hay valores negativos, los dejamos como NaN
df.loc[df["dias_fuera"] < 0, "dias_fuera"] = None


In [20]:
output_path = "/Users/agustin/Documents/Agustin_2025/Juan Colon/data/lesiones_clean.csv"
df.to_csv(output_path, index=False, encoding="utf-8")
print(f"✅ Archivo limpio guardado en:\n{output_path}")


✅ Archivo limpio guardado en:
/Users/agustin/Documents/Agustin_2025/Juan Colon/data/lesiones_clean.csv
